# Geoscripting Exercise 12 GISRSArtists
## 22 January 2019

Imports

In [4]:
from geopy.geocoders import Nominatim
from owslib.wcs import WebCoverageService
from owslib.wfs import WebFeatureService
from python.createBoundingBox import createBoundingBox
from python.writeGeotiffToFile import writeGeotiffToFile
from python.getGeotiffFromWebCoverageService import getGeotiffFromWebCoverageService
from python.getDataFromWebFeatureService import getDataFromWebFeatureService
from python.calculateBuildingVolume import calculateBuildingVolume
from python.visualizeBuildingVolume import visualizeBuildingVolume
from python.geocodePlacenameToCoordinates import geocodePlacenameToCoordinates
from python.reproject import reproject

ModuleNotFoundError: No module named 'geopy'

In [10]:
# get coordinates of Wageningen and reproject to RD New
wageningen = geocodePlacenameToCoordinates('Wageningen University')
x, y = reproject(wageningen, '4326', '28992')

# create bounding box around Wageningen
bbox = createBoundingBox(x, y)


(173494.1578792833, 443633.60329471016, 174494.1578792833, 444633.60329471016)
